# Huggingface Token setzen

In [1]:
import os
hf_token = "hf_Token"
#hf_token = os.getenv("HF_TOKEN")

# Git Repo per HTTPs Clonen

In [2]:
!git clone https://github.com/qvest-digital/Workshop_Agentic_AI.git

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Cloning into 'Workshop_Agentic_AI'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 216 (delta 96), reused 193 (delta 73), pack-reused 0 (from 0)
Receiving objects: 100% (216/216), 365.80 KiB | 5.38 MiB/s, done.
Resolving deltas: 100% (96/96), done.


# Pfad setzen

In [3]:
#SYSTEM_PATH = "/home/simon/Workshop_Agentic_AI"
SYSTEM_PATH = "./Workshop_Agentic_AI"

# Requirements installieren

In [4]:
!pip install -r "$SYSTEM_PATH/requirements.txt"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


# Speicherfragmentierung minimieren

In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Herstellen der Vorbedingungen aus Teil 1:

## Modell laden: lokal oder von Hugging Face

### Funktion
- Lädt ein Llama-3.1-Instruct-Modell entweder:
  - lokal aus dem Ordner ./models/llama-3.1-8b, wenn der Pfad vorhanden ist, oder
  - online von Hugging Face mit der Modell-ID meta-llama/Llama-3.1-8B-Instruct, und speichert es anschließend lokal ab.
- Die Umgebungsvariablen werden mit load_dotenv() aus einer .env-Datei geladen, u. a. das Hugging-Face-Token.

### Inputs
- Dateisystem:
  - Existenz von MODEL_PATH (./models/llama-3.1-8b).
- Umgebungsvariable:
  - HF_TOKEN (wird mit os.getenv("HF_TOKEN") gelesen) – persönliches Zugriffstoken für Hugging Face.
- Hyperparameter:
  - MODEL_ID: gibt die zu ladende Modell-ID an.
- Hardware:
  - device_map="auto" versucht automatisch, GPU(s) oder CPU sinnvoll zu nutzen.
  - torch_dtype="auto" bzw. dtype="auto" lässt das Modell selbst einen sinnvollen Datentyp wählen (z. B. bfloat16 oder float16).

### Outputs
- Globale Python-Variablen:
  - tokenizer: Instanz von AutoTokenizer, konfiguriert für das Llama-3.1-Modell.
  - model: Instanz von AutoModelForCausalLM, bereit für Textgenerierung.

In [6]:
from dotenv import load_dotenv
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

load_dotenv()

MODEL_PATH = f"{SYSTEM_PATH}/models/llama-3.1-8b"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

if os.path.exists(MODEL_PATH):
    print("Lade Modell lokal …")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained( MODEL_PATH, torch_dtype="auto", device_map="auto" )

else:
    print("Lade Modell von Hugging Face …")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, token=hf_token, dtype="auto", device_map="auto")

    # lokal speichern
    model.save_pretrained(MODEL_PATH)
    tokenizer.save_pretrained(MODEL_PATH)



Lade Modell von Hugging Face …


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pipeline erstellen
### Funktion
- Baut eine Hugging-Face-pipeline für Textgenerierung auf Basis des zuvor geladenen Modells und Tokenizers.
- Diese Pipeline kapselt:
  - Tokenisierung,
  - das Aufrufen des Modells,
  - und das Zurückkonvertieren der Token in Text.

### Inputs
- model: Causal-Language-Model (AutoModelForCausalLM), im vorherigen Block geladen.
- tokenizer: passender Tokenizer zu diesem Modell (AutoTokenizer).
- Task-Typ: "text-generation" – legt fest, dass es sich um eine generative Textaufgabe handelt.

### Outputs
- Variable:
  - llm: eine aufrufbare Pipeline-Instanz.

### Rückgabewert bei Aufruf von llm(...):
```python
[
  {
    "generated_text": "Vollständiger generierter Text (inkl. Prompt oder abhängig von den Parametern)"
  }
]
```

In [7]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


## Einfache Chat-Funktion zur Verfügung stellen

### Funktion
- Stellt ein vereinfachtes Chat-Interface zur Verfügung, das direkt mit einem String arbeitet.
- Die Funktion:
  - ruft intern die llm-Pipeline auf,
  - übergibt alle relevanten Generationsparameter,
  - und gibt am Ende nur den reinen generierten Text zurück (str statt verschachtelte Struktur).

### Inputs
- Pflichtparameter:
  - prompt: str – der Eingabetext an das Modell.
- Optionale Parameter zum Experimentieren
  - max_new_tokens: maximale Anzahl neu zu generierender Token.
  - temperature: steuert die Zufälligkeit (0 ≈ deterministischer, >0 zufälliger).
  - top_k: Sampling nur aus den k wahrscheinlichsten Token.
  - top_p: „Nucleus Sampling“ – Auswahl aus der kleinsten Masse der wahrscheinlichsten Token, deren Summe ≥ p ist.
  - typical_p: Bevorzugt Token mit kontexttypischer Wahrscheinlichkeit statt nur der höchsten.
  - repetition_penalty: >1.0 bestraft Wiederholungen.
  - length_penalty: Steuert, ob Beam Search kürzere oder längere Sequenzen bevorzugt.
  - no_repeat_ngram_size: Verhindert die Wiederholung identischer n-Gramme.
  - num_beams, num_beam_groups, diversity_penalty: Parameter für Beam Search (systematisches Durchsuchen mehrerer Kandidaten).
  - early_stopping: beendet Beam Search frühzeitig, wenn bestimmte Kriterien erfüllt sind.

### Outputs
- Rückgabewert:
  - str: der vom Modell generierte Antworttext (out[0]["generated_text"] ohne führende/trailing Leerzeichen).

### Typische Verwendung:
- llama_chat("Erkläre mir kurz, was ein LLM ist.")

In späteren Zellen wird statt eines rohen Prompts ein Chat-Prompt übergeben, der mit build_chat_prompt erzeugt wird.

In [8]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.01,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

Zeit zum Experimentieren (5 - 10 min.)

# Herstellen der Vorbedingungen aus Teil 3:

## MCP-Server als Subprozess starten

### Funktion
- Startet einen MCP-Server als separaten Python-Prozess über subprocess. Popen.
- Setzt PYTHONPATH so, dass mcp_server.mcp_tools.mcp.server als Modul importierbar ist.
- Hält die Projektpfade und Host/Port-Konfiguration zusammen.

### Input
- Konstanten:
    - PROJECT_ROOT / project_root: Pfad zum Projekt (SYSTEM_PATH).
    - MODULE_NAME: mcp_server.mcp_tools.mcp.server.
    - HOST, PORT: 127.0.0.1:8765.

### Umgebungsvariablen:
- Kopie von os.environ, erweitert um PYTHONPATH

### Output
- Subprozess proc:
    - MCP-Server läuft im Hintergrund unter der angegebenen PID.

### Konsole:
- Server-PID: ... zur Kontrolle, dass der Server wirklich gestartet wurde.

In [9]:
import subprocess, sys, os
from pathlib import Path

# -------------------------------------------------------------------
# Konfiguration
# -------------------------------------------------------------------
PROJECT_ROOT = Path(SYSTEM_PATH).resolve()
HOST = "127.0.0.1"
PORT = 8765
MCP_URL = f"http://{HOST}:{PORT}/mcp"

MODULE_NAME = "mcp_server_math.mcp_tools_math.mcp.server"

env = os.environ.copy()
env["PYTHONPATH"] = str(PROJECT_ROOT)  # falls nötig, damit mcp_tools_travel importierbar ist

In [ ]:
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

In [9]:
cmd = [
    sys.executable,
    "-m",
    MODULE_NAME,
    "--host",
    HOST,
    "--port",
    str(PORT),
]

proc = subprocess.Popen(cmd, cwd=str(PROJECT_ROOT), env=env)
print("Server-PID:", proc.pid)


Server-PID: 904


## MCP-Tools mit Metadaten vom Server abholen

### Funktion
- Baut eine Client-Verbindung zum MCP-Server via HTTP-Transport auf.
- Führt session.initialize() aus.
- Ruft session.list_tools() auf und gibt die komplette Liste der Tools inkl. Metadaten zurück.
- Hat ein Retry-Verhalten, falls der Server noch nicht bereit ist.

### Input
- url: str = MCP_URL – z. B. http://127.0.0.1:8765/mcp.
- retries: int = 50, delay: float = 0.1 – wie oft und wie lange gewartet wird, bis der Server erreichbar ist.

### Output
- Rückgabewert von fetch_tools_with_metadata(...):
    - tools_resp.tools: Liste von Tool-Objekten (inkl. Name, Beschreibung, Input-/Output-Schema).
- Bei Fehlern nach allen Retries:
    - RuntimeError("Keine Verbindung zum MCP-Server möglich: ...").

In [10]:
import asyncio
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

async def fetch_tools_with_metadata(
    url: str = MCP_URL,
    retries: int = 50,
    delay: float = 0.1,
):
    last_err = None
    for _ in range(retries):
        try:
            async with streamable_http_client(url) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as session:
                    await session.initialize()
                    tools_resp = await session.list_tools()
                    return tools_resp.tools  # komplette Objekte, nicht nur Namen
        except Exception as e:
            last_err = e
            await asyncio.sleep(delay)
    raise RuntimeError(f"Keine Verbindung zum MCP-Server möglich: {last_err}")

## Roh-Toolliste inspizieren

### Funktion
- Führt await fetch_tools_with_metadata() aus und speichert das Ergebnis.
- Gibt die unverarbeiteten Tool-Objekte auf der Konsole aus.

### Input
- MCP-Server muss laufen.
- Kein weiterer Parameter, MCP_URL wird aus dem globalen Kontext genommen.

### Output
- Variable tool_names_with_meta – Liste der Tool-Objekte (Client-Strukturen).

### Konsole:
- Debug-Print der Liste, z. B. <Tool name='geocode' ...> etc.

In [11]:
tool_names_with_meta = await fetch_tools_with_metadata()
print(f"tool_names_with_meta: {tool_names_with_meta}")

INFO:     127.0.0.1:43632 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43648 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:43662 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43668 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43680 - "DELETE /mcp HTTP/1.1" 200 OK
tool_names_with_meta: [Tool(name='addition', title=None, description='Addiert a + b', inputSchema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'additionArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'additionOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None), Tool(name='subtraktion', title=None, description='Subtrahiert a - b', inputSchema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'subtraktionArguments', 'type'

[01/31/26 10:40:56] INFO     Created new          ]8;id=536174;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=148729;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             9d71664d9db946d6be77                               
                             a00bf9ced544                                       
                    INFO     Processing request of type            ]8;id=458175;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=110693;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

## Tool-Metadaten für Prompts formatieren: format_tools_for_prompt

### Funktion
- Nimmt die MCP-Tool-Objekte und erzeugt eine lesbare Textbeschreibung für den System-Prompt:
    - Name, Beschreibung,
    - JSON-Input-Schema,
    - JSON-Output-Schema.
- Ist robust genug, um sowohl Dicts als auch Objekte mit Attributen (.name, .description, .input_schema, .output_schema) zu verarbeiten.

### Input
- tools: List[Any] – Liste der Tools, z. B. aus fetch_tools_with_metadata.

### Output
- str: Formatierte Version der Tools.

In [12]:
from typing import List, Any
import json

def format_tools_for_prompt(tools: List[Any]) -> str:
    """
    Macht aus den MCP-Tool-Objekten einen lesbaren Katalog für das System-Prompt.
    Funktioniert sowohl, wenn die Tools Dicts sind, als auch, wenn sie Attribute haben.
    """
    lines = []
    for t in tools:
        is_dict = isinstance(t, dict)

        # Name
        name = getattr(t, "name", None)
        if name is None and is_dict:
            name = t.get("name")

        # Beschreibung
        desc = getattr(t, "description", None)
        if desc is None and is_dict:
            desc = t.get("description", "")
        if desc is None:
            desc = ""

        # Input-Schema
        input_schema = (
            getattr(t, "input_schema", None)
            or getattr(t, "inputSchema", None)
            or (t.get("input_schema") if is_dict else None)
            or (t.get("inputSchema") if is_dict else None)
            or {}
        )

        # Output-Schema (neu)
        output_schema = (
            getattr(t, "output_schema", None)
            or getattr(t, "outputSchema", None)
            or (t.get("output_schema") if is_dict else None)
            or (t.get("outputSchema") if is_dict else None)
            or {}
        )

        lines.append(
            f"- Name: {name}\n"
            f"  Beschreibung: {desc}\n"
            f"  Eingabe-Schema (JSON): {json.dumps(input_schema, ensure_ascii=False)}\n"
            f"  Ausgabe-Schema (JSON): {json.dumps(output_schema, ensure_ascii=False)}"
        )

    return "\n\n".join(lines)

## Formatierten Tool-Katalog ansehen

### Funktion
- Ruft format_tools_for_prompt(tool_names_with_meta) auf.
- Gibt den formatierten Katalog auf der Konsole aus, damit du siehst, welche Tools der Server anbietet.

### Input
- tool_names_with_meta aus dem vorherigen Schritt.

### Output
- tool_names_with_meta_formated: str – fertiger Tool-Katalog-String.

### Konsolenausgabe
- Konsolenausgabe des Strings (für menschlichen Check).

In [13]:
tool_names_with_meta_formated = format_tools_for_prompt(tool_names_with_meta)
print(tool_names_with_meta_formated)

- Name: addition
  Beschreibung: Addiert a + b
  Eingabe-Schema (JSON): {"properties": {"a": {"title": "A", "type": "number"}, "b": {"title": "B", "type": "number"}}, "required": ["a", "b"], "title": "additionArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"properties": {"result": {"title": "Result", "type": "number"}}, "required": ["result"], "title": "additionOutput", "type": "object"}

- Name: subtraktion
  Beschreibung: Subtrahiert a - b
  Eingabe-Schema (JSON): {"properties": {"a": {"title": "A", "type": "number"}, "b": {"title": "B", "type": "number"}}, "required": ["a", "b"], "title": "subtraktionArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"properties": {"result": {"title": "Result", "type": "number"}}, "required": ["result"], "title": "subtraktionOutput", "type": "object"}

- Name: multiplikation
  Beschreibung: Multipliziert a * b
  Eingabe-Schema (JSON): {"properties": {"a": {"title": "A", "type": "number"}, "b": {"title": "B", "type": "number"}}, "require

## Prompt-Builder mit sauberer Trennung von Tool-Agent- und No-Tool-Modus

### Funktion
- Erzeugt einen Chat-Prompt, der zwei Betriebsarten explizit trennt:
  - **Tool-Agent-Modus** (JSON-only): Modell soll Tool-Pläne oder Final/Error als *genau ein JSON-Objekt* ausgeben.
  - **No-Tool-Modus** (Final-only): Modell darf keine Tools nutzen und kein JSON ausgeben; es soll das finale Ergebnis
    ausschließlich aus der Tool-Historie ableiten.
- Aggregiert Systemprompt, optionalen Dialogverlauf, optionale Tool-Historie und die aktuelle Nutzerfrage in eine
  konsistente Nachrichtenliste.

### Inputs
- Pflichtparameter:
  - `user_prompt: str` – aktuelle Nutzerfrage bzw. aktueller Ausdruck.
- Optionale Parameter:
  - `system_prompt: Optional[str]` – Agentenregeln/Systeminstruktionen (insb. für Tool-Agent-Modus).
  - `history: Optional[List[Tuple[str, str]]]` – Dialogverlauf als (User, Assistant)-Paare.
  - `tool_history: Optional[List[Dict[str, Any]]]` – strukturierte Historie früherer Tool-Aufrufe inkl. `replace_target`.
  - `allow_tools: bool` – Schalter zwischen Tool-Agent-Modus (`True`) und No-Tool-Modus (`False`).

### Modus-Semantik
- `allow_tools=True`:
  - Prompt lässt Tool-Agentenregeln gelten (sofern `system_prompt` gesetzt).
  - Erwartete Ausgabe: strikt maschinenlesbar (JSON-only), geeignet für Plan-Parsing.
- `allow_tools=False`:
  - Fügt einen zusätzlichen System-Override ein, der:
    - Tool-Nutzung verbietet,
    - JSON-Ausgabe verbietet,
    - und eine Minimal-Ausgabe erzwingt (entweder reine Zahl oder exakt `invalid_expression`).
  - Modell soll das Ergebnis deterministisch aus `tool_history` ableiten, indem es einen Eintrag sucht,
    dessen `replace_target` exakt dem aktuellen Ausdruck entspricht.

### Nachrichtenstruktur
- System:
  - optional `system_prompt`.
  - optional No-Tool-Override (bei `allow_tools=False`).
- Verlauf:
  - frühere User/Assistant-Nachrichten in Reihenfolge.
- System:
  - optional Tool-Historie, serialisiert als JSON.
- User:
  - aktuelle Nutzerfrage als letzte Nachricht.

### Outputs
- Rückgabewert:
  - `str` – gerenderter Chat-Prompt via Chat-Template (`add_generation_prompt=True`), bereit für die direkte LLM-Übergabe.

### Typische Verwendung
- Agentische Pipelines mit **zweiphasigem** Ablauf:
  - Phase 1: planen/ausführen mit Tools (JSON-only).
  - Phase 2: finale Ausgabe ohne Tools (aus Tool-Historie).
- Prompt-Debugging und kontrollierte Evaluationsläufe mit klaren Output-Verträgen.

In [14]:
from typing import Any, Dict, List, Optional, Tuple
import json

def build_chat_prompt_with_rag_and_tools(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
    allow_tools: bool = True,
) -> str:
    """
    Prompt-Builder, der Tool-Agent-Modus (JSON-only) und No-Tool-Modus (Final-Antwort aus Tool-History) sauber trennt.

    WICHTIG:
    - allow_tools=True  => Tool-Agent-Modus: Ausgabe MUSS exakt ein JSON-Objekt sein (Toolcall ODER Final/Error).
    - allow_tools=False => No-Tool-Modus: Keine Tools, kein JSON-Zwang; das Modell soll das finale Resultat aus
                          Tool-History liefern (z.B. "38.0") oder "invalid_expression".
    """
    messages: List[Dict[str, str]] = []

    # 1) System-Prompt (Tool-Agent Regeln, wenn vorhanden)
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    # 1b) Override für Phase 2: NO-TOOL / FINAL-AUSGABE
    if not allow_tools:
        messages.append(
            {
                "role": "system",
                "content": (
                    "WICHTIG: In dieser Runde darfst du KEINE Tools aufrufen.\n"
                    "Du darfst KEIN JSON ausgeben.\n"
                    "Nutze ausschließlich die Informationen aus der Tool-Historie.\n\n"
                    "AUFGABE:\n"
                    "- Bestimme das finale Ergebnis, indem du in der Tool-Historie nach einem Eintrag suchst, "
                    "dessen replace_target exakt dem aktuellen User-Ausdruck entspricht.\n"
                    "- Falls vorhanden, gib NUR die Zahl (als Fließtext, ohne zusätzliche Wörter) aus, z.B.:\n"
                    "  38.0\n"
                    "- Falls nicht vorhanden, gib exakt aus:\n"
                    "  invalid_expression\n\n"
                    "REGELN:\n"
                    "- Keine Erklärungen.\n"
                    "- Keine Zwischenschritte.\n"
                    "- Keine zusätzlichen Tokens außer der Ergebniszahl oder 'invalid_expression'."
                ),
            }
        )

    # 2) Verlauf (User/Assistant)
    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # 3) Tool-History (falls vorhanden)
    if tool_history:
        messages.append(
            {
                "role": "system",
                "content": "Tool-Historie:\n" + json.dumps(tool_history, ensure_ascii=False),
            }
        )

    # 4) aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt

# Teil 4: Voll agentisches MCP – der Mathematik-Assistent denkt und handelt selbst

In Teil 4 machen wir aus dem „Half-Agent“ aus Teil 3 jetzt einen echten, voll agentischen Workflow:
- Das LLM plant eine Tool-Kette (inkl. Parallelität).
- Es referenziert frühere Tool-Outputs über $ref.
- Ein Steuerloop (agentic_run_to_final_answer) führt Schritt für Schritt alle geplanten Tools aus, aktualisiert die Historie und lässt das Modell bei Bedarf re-planen, bis eine finale Antwort entsteht.

Konzeptionell lehnen wir uns an Tool-/Plan-basierte Agenten wie ReAct und ähnliche Frameworks an, die iterativ zwischen „Denken“, „Handeln (Tool)“ und „Beobachten“ wechseln.

## System-Prompt-Erzeugung für einen deterministischen Mathe-Tool-Agenten

### Funktion
- Erzeugt einen umfassenden System-Prompt, der das Verhalten eines strikt tool-basierten Mathe-Agenten festlegt.
- Integriert einen Tool-Katalog in den Prompt, sodass das Modell die verfügbaren Tools und deren Signaturen kennt.
- Zielt auf maximale Deterministik und Nachvollziehbarkeit durch harte Regeln und ein festes JSON-Outputformat.

### Inputs
- Pflichtparameter:
  - `tools: List[Any]` – Liste verfügbarer Tools, die im Prompt als Tool-Katalog formatiert werden.

### Prompt-Inhalt
- Definiert eine strikte Rolle:
  - Der Agent darf **niemals selbst rechnen**.
  - Jede Operation muss über ein Tool erfolgen.
- Legt die erlaubten Operationen und Einschränkungen fest (z. B. keine Klammern, keine Potenzen).
- Erzwingt Operatorpräzedenz (*/ vor +-; jeweils links-nach-rechts).
- Definiert Referenzregeln (`$ref`) für die Verkettung von Tool-Ergebnissen.
- Beschreibt eine Zustands-Transformation über `replace_target` anhand der Tool-Historie (Caching/Ersetzung statt Neubewertung).
- Legt die Strategie zur Auswahl der nächsten Operation fest (erste offene Operation gemäß Präzedenz, exaktes Teilstückformat).
- Erzwingt ein einziges JSON-Objekt als Ausgabe ohne Zusatztext/Markdown.
- Enthält spezifizierte Fehlerausgaben für ungültige Ausdrücke und Division durch Null.

### Outputs
- Rückgabewert:
  - `str` – finaler System-Prompt als String, inklusive eingebettetem Tool-Katalog.

### Typische Verwendung
- Planner-LLM in agentischen Pipelines, die mathematische Ausdrücke zuverlässig in Tool-Pläne übersetzen sollen.
- Evaluation deterministischer Tool-Agenten mit reproduzierbaren Regeln und maschinenlesbarem Output.
- Absicherung gegen Halluzinationen durch harte Verbote (kein Rechnen, kein Freitext, nur JSON).

In [15]:
from typing import Any, List

def build_agent_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)

    return f"""
Du bist ein deterministischer MATHE-Tool-Agent.

Deine Aufgabe ist es, arithmetische Ausdrücke korrekt zu verarbeiten.
Du darfst NIEMALS selbst rechnen.
Jede mathematische Operation MUSS über ein Tool erfolgen.

====================================================================
ERLAUBTE OPERATIONEN
====================================================================

- Addition (+)
- Subtraktion (-)
- Multiplikation (*)
- Division (/)

Nicht erlaubt:
- Klammern
- negative Zahlen
- Potenzen
- Funktionen
- algebraische Umformungen

====================================================================
OPERATORPRÄZEDENZ
====================================================================

1. Multiplikation und Division (links → rechts)
2. Addition und Subtraktion (links → rechts)

Diese Reihenfolge darf niemals verletzt werden.

====================================================================
GRUNDREGEL
====================================================================

Ein Ergebnis existiert für dich nur, wenn es aus einem Tool stammt.
Alles andere gilt als unbekannt.

====================================================================
$ref-REGELN
====================================================================

#<<Bitte ergänzen>>

====================================================================
ZUSTANDSTRANSFORMATION
====================================================================

Vor jeder Planung:

- Ersetze im aktuellen Ausdruck alle bekannten replace_target-Substrings
  durch die exakten Tool-Resultate aus der Tool-Historie.
- Verwende exakt die String-Repräsentation der Tool-Ergebnisse.
- Rekonstruiere oder errate niemals Werte.

Wenn der Ausdruck danach exakt eine einzelne Zahl ist:
→ gehe sofort in den FINAL-MODUS.

====================================================================
AUSWAHL DER NÄCHSTEN OPERATION
====================================================================

- Wähle immer die erste noch offene Operation gemäß Präzedenz.
- Das Teilstück muss exakt die Form haben:
  <number> <operator> <number>

====================================================================
TOOL-PLANUNG
====================================================================

#<<Bitte ergänzen>>

====================================================================
CACHE
====================================================================

#<<Bitte ergänzen>>

====================================================================
OUTPUTFORMAT
====================================================================

Du gibst IMMER genau EIN JSON-Objekt aus.
Kein zusätzlicher Text. Kein Markdown.

Format:

{{
  "steps": [
    {{
      "description": "<kurze Beschreibung ohne Ergebnis>",
      "tools": [
        {{
          "id": "<eindeutige-id>",
          "name": "<tool-name>",
          "arguments": {{
            "a": <number oder $ref>,
            "b": <number oder $ref>
          }},
          "replace_target": "<exakter substring '<a> <op> <b>'>"
        }}
      ]
    }}
  ],
  "final": null | "<zahl als String>" | {{"$ref":"<tool-id>[.<pfad>]"}} | "<error_code>"
}}

====================================================================
FEHLER
====================================================================

Ungültiger Ausdruck:
{{
  "steps": [],
  "final": "invalid_expression"
}}

Division durch 0 (literal erkennbar):
{{
  "steps": [],
  "final": "division_by_zero"
}}

TOOLS:
{tool_catalog}

""".strip()

## Agent-System-Prompt bauen

### Funktion
- Ruft build_agent_system_prompt(tool_names_with_meta) auf, um den finalen Prompt-String zu erzeugen.

### Input
- tool_names_with_meta – Toolmetadaten, wie zuvor per fetch_tools_with_metadata() geladen.

### Output
- agent_system_prompt: str – wird später in allen agentischen Calls verwendet.

### Konsole
- Ausgabe von agent_system_prompt

In [16]:
agent_system_prompt = build_agent_system_prompt(tool_names_with_meta)
print(agent_system_prompt)

Du bist ein deterministischer MATHE-Tool-Agent.

Deine Aufgabe ist es, arithmetische Ausdrücke korrekt zu verarbeiten.
Du darfst NIEMALS selbst rechnen.
Jede mathematische Operation MUSS über ein Tool erfolgen.

ERLAUBTE OPERATIONEN

- Addition (+)
- Subtraktion (-)
- Multiplikation (*)
- Division (/)

Nicht erlaubt:
- Klammern
- negative Zahlen
- Potenzen
- Funktionen
- algebraische Umformungen

OPERATORPRÄZEDENZ

1. Multiplikation und Division (links → rechts)
2. Addition und Subtraktion (links → rechts)

Diese Reihenfolge darf niemals verletzt werden.

GRUNDREGEL

Ein Ergebnis existiert für dich nur, wenn es aus einem Tool stammt.
Alles andere gilt als unbekannt.

$ref-REGELN

- Toolargumente dürfen entweder:
  - literale Zahlen
  - oder ein $ref-Objekt sein: {"$ref":"<tool-id>[.<pfad>]"}

- Erlaubter Pfad für Mathe-Tools:
  - ".result"

- {"$ref":"<tool-id>"} ist ein Shortcut für {"$ref":"<tool-id>.result"}.
- Beide Varianten sind erlaubt und gleichwertig.

- $ref muss immer als Ob

## Erster agentischer Call: Was ist 6.0 + 4.0 * 8.0?

### Kontext
- Demonstriert den Aufbau eines vollständigen Chat-Prompts inklusive Systemkontext und Nutzerfrage.
- Zeigt, ob die generierung des Plans funktioniert.

### Ablauf
- Definiert eine einfache arithmetische Nutzerfrage als String.
- Initialisiert leere Strukturen für Gesprächs- und Tool-Historie.
- Erzeugt einen Chat-Prompt aus:
  - System-Prompt (Agentenregeln),
  - aktueller Nutzerfrage,
  - optionalem Dialogverlauf,
  - optionaler Tool-Historie.
- Gibt den generierten Prompt explizit aus, um dessen Struktur nachvollziehen zu können.
- Übergibt den Prompt direkt an das LLM und gibt die Rohantwort aus.

### Inputs
- `system_prompt` – globaler Systemkontext für das Modell.
- `user_prompt` – aktuelle Nutzerfrage.
- `history` – bisheriger Dialogverlauf (hier leer).
- `tool_history` – Historie früherer Tool-Aufrufe (hier nicht gesetzt).

### Outputs
- Konsolenausgabe des vollständigen Prompts.
- Konsolenausgabe des Plans.

In [17]:
user_input = f"Was ist 6.0 + 4.0 * 8.0?"
tool_history = None
history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=agent_system_prompt,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein deterministischer MATHE-Tool-Agent.

Deine Aufgabe ist es, arithmetische Ausdrücke korrekt zu verarbeiten.
Du darfst NIEMALS selbst rechnen.
Jede mathematische Operation MUSS über ein Tool erfolgen.

ERLAUBTE OPERATIONEN

- Addition (+)
- Subtraktion (-)
- Multiplikation (*)
- Division (/)

Nicht erlaubt:
- Klammern
- negative Zahlen
- Potenzen
- Funktionen
- algebraische Umformungen

OPERATORPRÄZEDENZ

1. Multiplikation und Division (links → rechts)
2. Addition und Subtraktion (links → rechts)

Diese Reihenfolge darf niemals verletzt werden.

GRUNDREGEL

Ein Ergebnis existiert für dich nur, wenn es aus einem Tool stammt.
Alles andere gilt als unbekannt.

$ref-REGELN

- Toolargumente dürfen entweder:
  - literale Zahlen
  - oder ein $ref-Objekt sein: {"$ref":"<tool-id>[.<pfad>]"}

- Erlaubter Pfad für Mathe-Tools:
  - ".result"

- {"$ref":"<tool

# Im Anschluss werden wir die bearbeitung dieses Plans automatisieren

## Parsing eines JSON-Plans aus LLM-Output

### Funktion
- Extrahiert und parst einen strukturierten Ausführungsplan (JSON) aus dem Rohtext eines Modells.
- Ist robust gegen „Text“ um das JSON herum, indem gezielt das erste vollständige JSON-Objekt herausgeschnitten wird.
- Validiert minimale Schemaanforderungen, damit nachgelagerte Komponenten zuverlässig arbeiten können.

### Inputs
- Pflichtparameter:
  - `output_str: str` – Rohausgabe des Planungs-LLMs (kann Text + JSON enthalten).

### Extraktionslogik
- Fängt leere oder triviale Ausgaben früh ab.
- Sucht die erste öffnende geschweifte Klammer `{` als Startpunkt des JSON-Kandidaten.
- Bestimmt das Ende des JSON-Objekts durch kontrolliertes Klammer-Counting:
  - berücksichtigt dabei String-Literale (`"..."`), damit Klammern innerhalb von Strings nicht fälschlich gezählt werden,
  - berücksichtigt Escape-Sequenzen (z. B. `\"`), um String-Grenzen korrekt zu erkennen.
- Schneidet den ermittelten Bereich als „trimmed JSON candidate“ aus und protokolliert ihn für Debugging.

### Parsing & Validierung
- Parst den ausgeschnittenen Kandidaten mittels `json.loads`.
- Hebt Parsing-Fehler mit einer aussagekräftigen Fehlermeldung an, inklusive Kandidat und Original-Exception.
- Prüft ein Minimal-Schema:
  - Plan muss `steps` und `final` enthalten.
  - `steps` muss eine Liste sein.

### Outputs
- Rückgabewert:
  - `Dict[str, Any]` – geparster und validierter Plan als Dictionary.

In [18]:
import json
from typing import Any, Dict

def agentic_parse_model_plan(output_str: str) -> Dict[str, Any]:
    #print("[DEBUG] raw output:", repr(output_str))

    # Leer- oder Nonsens-Output früh abfangen
    if not output_str or not output_str.strip():
        raise ValueError("Model output is empty, cannot parse JSON plan.")

    # Die erste öffnende Klammer suchen
    start = output_str.find("{")
    if start == -1:
        raise ValueError(f"No JSON object found in model output: {output_str}")

    s = output_str

    # Jetzt von 'start' an die passende schließende Klammer suchen
    depth = 0
    in_string = False
    escape = False
    end = None  # <- no Optional annotation needed

    for i, ch in enumerate(s[start:], start=start):
        if in_string:
            # Innerhalb eines Strings
            if escape:
                # nächstes Zeichen ist escaped, einfach überspringen
                escape = False
            elif ch == "\\":
                escape = True
            elif ch == '"':
                in_string = False
        else:
            # Außerhalb von Strings
            if ch == '"':
                in_string = True
            elif ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    end = i
                    break

    trimmed = s[start:end + 1] if end is not None else s[start:]
    print("[DEBUG] trimmed JSON candidate:", trimmed)

    try:
        plan = json.loads(trimmed)
    except json.JSONDecodeError as e:
        raise ValueError(
            f"Failed to parse JSON plan from model output. Error: {e}. "
            f"Trimmed candidate: {trimmed}"
        ) from e

    if "steps" not in plan or "final" not in plan:
        raise ValueError("Model output must contain 'steps' and 'final'.")

    if not isinstance(plan["steps"], list):
        raise ValueError("'steps' must be a list.")

    return plan

## Auflösung einzelner Referenz-Argumente (`$ref`)

### Funktion
- Löst einzelne Argumentwerte auf, die als Referenz auf Nutzerkontext oder Tool-Ergebnisse definiert sind.
- Unterstützt mehrere syntaktische Varianten für Referenzen, um flexible Plan-Spezifikationen zu ermöglichen.
- Gibt bei nicht-referenzierten Werten den Originalwert unverändert zurück.

### Unterstützte Referenzformate
- Objektform:
  - `{"$ref": "tool_id.key.subkey"}`
- Stringform:
  - `"$ref:tool_id.key.subkey"`
- Zugriff auf Nutzerkontext:
  - `"$ref:user.raw"`

### Auflösungslogik
- Extrahiert zunächst den Referenzstring (`ref_str`), falls vorhanden.
- Zerlegt die Referenz in einen Wurzelbezeichner (`root`) und einen Zugriffspfad.
- Bestimmt die Datenquelle:
  - `user` → Zugriff auf den Nutzerkontext.
  - sonst → Zugriff auf gespeicherte Tool-Ergebnisse anhand der Tool-ID.
- Traversiert den Zugriffspfad schrittweise:
  - unterstützt Dictionary-Zugriffe über Schlüssel,
  - unterstützt Listen-Zugriffe über numerische Indizes.
- Bricht mit aussagekräftigen Fehlermeldungen ab, wenn ein Pfad ungültig ist.

### Inputs
- `value: Any` – zu prüfender und ggf. aufzulösender Wert.
- `tool_results: Dict[str, Any]` – verfügbare Ergebnisse früherer Tool-Aufrufe.
- `user_ctx: Dict[str, Any]` – Nutzerkontext (z. B. rohe Eingabe, Metadaten).

### Outputs
- Rückgabewert:
  - `Any` – aufgelöster Wert oder der unveränderte Eingabewert, falls keine Referenz vorliegt.

In [19]:
def agentic_resolve_argument(
    value: Any,
    tool_results: Dict[str, Any],
    user_ctx: Dict[str, Any],
) -> Any:
    """
    Löst einzelne Argumentwerte auf. Unterstützt:
    - {"$ref": "geo_roma.lat"}
    - "$ref:geo_roma.lat"
    - "$ref:user.raw"
    """
    ref_str: Optional[str] = None

    # Fall 1: Objekt mit "$ref"
    if isinstance(value, dict) and "$ref" in value and isinstance(value["$ref"], str):
        ref_str = value["$ref"]

    # Fall 2: String mit Präfix "$ref:"
    elif isinstance(value, str) and value.startswith("$ref:"):
        ref_str = value[len("$ref:") :]

    # Kein $ref → Wert unverändert zurückgeben
    if ref_str is None:
        return value

    # Jetzt ref_str auswerten, z.B. "geo_roma.lat" oder "user.raw"
    parts = ref_str.split(".")
    root = parts[0]
    path = parts[1:]

    # Quelle: User-Kontext
    if root == "user":
        current: Any = user_ctx
    else:
        # Quelle: Tool-Resultate
        if root not in tool_results:
            raise KeyError(f"Unknown tool id in $ref: {root}")
        current = tool_results[root]

    # Debug: einmal zeigen, was wir da wirklich haben
    print(f"[DEBUG] Resolving $ref '{ref_str}': root='{root}', initial_type={type(current)}, initial_value={current}")

    for p in path:
        # Numerische Indizes unterstützen (z.B. itineraries.0)
        if isinstance(current, list) and p.isdigit():
            current = current[int(p)]
        elif isinstance(current, dict) and p in current:
            current = current[p]
        else:
            # Noch mehr Debug, bevor wir crashen
            raise KeyError(
                f"Invalid path component '{p}' in $ref '{ref_str}' "
                f"(current_type={type(current)}, current_value={current})"
            )

    return current

## Auflösung von Tool-Argumenten mit Referenzen

### Funktion
- Löst Tool-Argumente rekursiv auf, bevor sie an ein externes Tool übergeben werden.
- Unterstützt Referenzen auf frühere Tool-Ergebnisse und den Nutzerkontext.
- Vereinheitlicht die Argumentstruktur unabhängig von Verschachtelungstiefe.

### Logik
- Erkennt Dictionaries, Listen und skalare Werte und behandelt sie rekursiv.
- Spezialfall:
  - Besteht ein Dictionary ausschließlich aus einem `$ref`-Eintrag, wird das gesamte Objekt
    durch das referenzierte Ergebnis ersetzt.
- Für reguläre Dictionaries:
  - Jedes Argument wird einzeln rekursiv aufgelöst.
- Für Listen:
  - Jedes Element wird rekursiv aufgelöst.
- Skalare oder nicht strukturierte Werte werden direkt über die Referenzauflösung verarbeitet.

### Inputs
- `arguments` – rohe Tool-Argumente (beliebig verschachtelt).
- `tool_results` – bisherige Tool-Ergebnisse, die als Referenzquelle dienen.
- `user_ctx` – Nutzerkontext (z. B. rohe Eingabe oder abgeleitete Werte).

### Outputs
- Rückgabewert:
  - vollständig aufgelöste Argumentstruktur, bereit zur Übergabe an ein Tool.

In [20]:
def agentic_resolve_tool_arguments(arguments, tool_results, user_ctx):
    if isinstance(arguments, dict):
        # Spezialfall: dict besteht NUR aus "$ref" → ersetze das ganze Dict durch das referenzierte Objekt
        if set(arguments.keys()) == {"$ref"} and isinstance(arguments["$ref"], str):
            return agentic_resolve_argument(arguments, tool_results, user_ctx)

        return {k: agentic_resolve_tool_arguments(v, tool_results, user_ctx) for k, v in arguments.items()}

    if isinstance(arguments, list):
        return [agentic_resolve_tool_arguments(v, tool_results, user_ctx) for v in arguments]

    return agentic_resolve_argument(arguments, tool_results, user_ctx)

## Ausführung eines einzelnen Planschritts mit Tool-Aufrufen

### Funktion
- Führt alle in einem Planschritt definierten Tool-Aufrufe sequenziell aus.
- Verbindet Plan-Spezifikation, Tool-Invocation und Ergebnisverwaltung in einer klaren Ausführungseinheit.
- Liefert strukturierte History-Einträge zur späteren Verwendung im Prompt oder zur Nachvollziehbarkeit.

### Inputs
- Pflichtparameter:
  - `step: Dict[str, Any]` – Planschritt mit Beschreibung und einer Liste von Tools
    (`{"description": ..., "tools": [...]}`).
  - `call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]]` – asynchrone Funktion zum Ausführen eines Tools.
  - `tool_results: Dict[str, Any]` – Sammelstruktur für Tool-Ergebnisse (wird in-place erweitert).
  - `user_ctx: Dict[str, Any]` – Nutzerkontext, z. B. rohe Nutzereingabe oder abgeleitete Werte.

### Ablauf
- Initialisiert eine leere Liste für Tool-History-Einträge.
- Iteriert über alle im Step definierten Tools in der vorgegebenen Reihenfolge.
- Löst referenzierte Argumente (`$ref`) anhand bisheriger Tool-Ergebnisse und des Nutzerkontexts auf.
- Ruft jedes Tool mit den aufgelösten Argumenten asynchron auf.
- Validiert und normalisiert das Tool-Ergebnis.
- Speichert das Ergebnis unter der Tool-ID für nachfolgende Schritte.
- Erzeugt pro Tool einen strukturierten History-Eintrag.

### Fehlerbehandlung
- Wirft einen `TypeError`, falls das Tool-Ergebnis nicht der erwarteten Dictionary-Struktur entspricht.
- Bricht den Ablauf mit einem `RuntimeError` ab, wenn ein Tool einen Fehler signalisiert.

### Outputs
- Rückgabewert:
  - `List[Dict[str, Any]]` – Liste strukturierter Tool-History-Einträge für Prompt-Aufbau oder Logging.

In [21]:
from typing import Callable, Awaitable

async def agentic_execute_step(
    step: Dict[str, Any],
    call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]],
    tool_results: Dict[str, Any],
    user_ctx: Dict[str, Any],
) -> List[Dict[str, Any]]:
    """
    Führt alle Tools in einem Step aus.
    - step: {"description": "...", "tools": [ {id, name, arguments}, ... ]}
    - call_tool_fn: deine MCP-Invoker-Funktion (z.B. call_mcp_tool)
    - tool_results: wird in-place mit neuen Ergebnissen gefüllt
    - user_ctx: z.B. {"raw": user_input}

    Rückgabe: Liste von History-Objekten für den Prompt (z.B. Tool-Logs).
    """

    history_entries: List[Dict[str, Any]] = []

    for tool in step.get("tools", []):
        tool_id = tool["id"]
        tool_name = tool["name"]
        raw_args = tool.get("arguments", {})
        replace_target = tool.get("replace_target")

        # $ref auflösen
        resolved_args = agentic_resolve_tool_arguments(raw_args, tool_results, user_ctx)

        # Tool aufrufen
        print(f"[{tool_id}] {tool_name} resolved arguments: {resolved_args}")
        result = await call_tool_fn(tool_name, resolved_args)
        print(f"toolcallresult: {result}")

        # Normalize (expect dict with keys isError/data from your agentic_call_mcp_tool)
        if not isinstance(result, dict):
            raise TypeError(f"Tool result must be dict, got {type(result)}: {result}")

        # History-Eintrag für Tools (kannst du an dein Format anpassen)

        if result.get("isError"):
            raise RuntimeError(f"Tool failed: {result}")

        res = result.get("data", result)

        # Ergebnis speichern für for Schleife bei parallelen Tools
        tool_results[tool_id] = res

        history_entries.append({
            "role": "tool",
            "tool_id": tool_id,
            "name": tool_name,
            "arguments": resolved_args,
            "result": res,
            "replace_target": replace_target,
        })

    return history_entries

## Auswahl des nächsten auszuführenden Planschritts

### Funktion
- Bestimmt den nächsten noch nicht ausgeführten Schritt in einem agentischen Ausführungsplan.
- Implementiert eine einfache, deterministische Auswahlstrategie ohne Reordering oder Heuristiken.

### Logik
- Iteriert über alle im Plan definierten Schritte in ihrer ursprünglichen Reihenfolge.
- Prüft für jeden Schritt, ob dessen Index bereits als ausgeführt markiert ist.
- Gibt den kleinsten Index zurück, der noch nicht in `executed_step_indices` enthalten ist.
- Liefert `None`, sobald alle Schritte abgearbeitet wurden.

### Inputs
- `plan: Dict[str, Any]` – strukturierter Ausführungsplan mit einer optionalen `steps`-Liste.
- `executed_step_indices: Set[int]` – Menge der bereits ausgeführten Schritt-Indizes.

### Outputs
- Rückgabewert:
  - `Optional[int]` – Index des nächsten auszuführenden Schritts oder `None`, wenn der Plan vollständig abgearbeitet ist.

In [22]:
from typing import Set

def agentic_find_next_step(plan: Dict[str, Any], executed_step_indices: Set[int]) -> Optional[int]:
    """
    Wählt den nächsten noch nicht ausgeführten Step.
    V1: simpel – der kleinste Index, der noch nicht in executed_step_indices ist.
    """
    for idx, _ in enumerate(plan.get("steps", [])):
        if idx not in executed_step_indices:
            return idx
    return None

## Prompt-Konstruktion mit RAG- und Tool-Kontext für agentische Planung

### Funktion
- Baut einen vollständigen Chat-Prompt für agentische Planungs- oder Reasoning-Phasen.
- Kombiniert Systemregeln, Tool-Historie und Gesprächsverlauf zu einer konsistenten Nachrichtenliste.
- Ziel ist es, dem LLM **maximalen, expliziten Kontext** für Planung und Entscheidungsfindung bereitzustellen.

### Inputs
- Pflichtparameter:
  - `tokenizer` – Tokenizer mit Chat-Template-Unterstützung.
  - `system_prompt: str` – globale Regeln und Instruktionen für den Agenten.
  - `user_prompt: str` – aktuelle Nutzerfrage.
- Optionale Parameter:
  - `history_pairs` – Liste aus `(user, assistant)`-Nachrichtenpaaren zur Kontextfortführung.
  - `tool_history` – strukturierte Historie vorheriger Tool-Aufrufe und Ergebnisse.
  - `retrieved_context` – externer Kontext aus Retrieval/RAG (z. B. Dokumente, Wissensschnipsel).

### Nachrichtenstruktur
- System-Nachrichten:
  - Agentenregeln (`system_prompt`).
  - Retrieval-Kontext, klar als externer Kontext markiert.
  - Tool-Historie, serialisiert als JSON zur maschinellen Interpretierbarkeit.
- Dialog-Nachrichten:
  - Frühere Nutzer- und Assistentenbeiträge in chronologischer Reihenfolge.
- Abschluss:
  - Aktuelle Nutzerfrage als letzte User-Nachricht.

### Outputs
- Rückgabewert:
  - `str` – gerenderter Chat-Prompt, bereit für die Übergabe an ein Chat-basiertes LLM.

### Design-Entscheidungen
- Explizite Trennung verschiedener Kontextquellen durch eigene System-Nachrichten.
- Verwendung von `add_generation_prompt=True`, um dem Modell klar zu signalisieren,
  dass die nächste Ausgabe vom Assistant zu erzeugen ist.
- Kein Tokenisieren an dieser Stelle, um Debugging und Prompt-Inspektion zu erleichtern.

In [23]:
def agentic_build_chat_prompt_with_rag_and_tools(
    tokenizer,
    system_prompt: str,
    user_prompt: str,
    history_pairs: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
    retrieved_context: Optional[str] = None,
) -> str:
    """
    Messages:
      - system: agent rules
      - system: retrieved context (optional)
      - system: tool history (optional, as JSON)
      - user/assistant: conversation history (optional)
      - user: current question
    """
    messages: List[Dict[str, str]] = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    if retrieved_context:
        messages.append({"role": "system", "content": f"Kontext (aus Retrieval):\n{retrieved_context}"})

    if tool_history:
        messages.append({"role": "system", "content": "Tool-Historie:\n" + json.dumps(tool_history, ensure_ascii=False)})

    if history_pairs:
        for u, a in history_pairs:
            messages.append({"role": "user", "content": u})
            messages.append({"role": "assistant", "content": a})

    messages.append({"role": "user", "content": user_prompt})

    # This is the crucial bit: add_generation_prompt=True
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

## Extraktion eines skalaren Ergebnisses aus Tool-Rückgaben

### Funktion
- Vereinfacht Tool-Ergebnisse auf einen skalaren Wert, sofern möglich.
- Dient als Normalisierungsschritt zwischen Tool-Ausführung und finaler Antwortformulierung.

### Logik
- Prüft, ob das übergebene Objekt ein Dictionary mit genau einem Eintrag ist.
- Erkennt das Muster eines typischen Tool-Ergebnisses (`{"result": <Wert>}`).
- Gibt in diesem Fall direkt den enthaltenen skalaren Wert zurück.
- Fällt andernfalls auf die unveränderte Rückgabe des ursprünglichen Objekts zurück.

### Inputs
- `x: Any` – beliebiges Objekt, typischerweise ein Tool-Ergebnis oder bereits ein skalarer Wert.

### Outputs
- Rückgabewert:
  - `Any` – extrahierter skalarer Wert oder das Originalobjekt, falls keine Vereinfachung möglich ist.

In [24]:
def _extract_scalar_result(x: Any) -> Any:
    # Wenn Tool-Resultate dicts sind wie {"result": 38.0}
    if isinstance(x, dict) and "result" in x and len(x) == 1:
        return x["result"]
    return x

## Prompt-Konstruktion für die finale Antwortformulierung (No-Tool-Phase)

### Funktion
- Erzeugt den finalen Chat-Prompt für die **Antwortformulierung ohne weitere Tool-Nutzung**.
- Dient ausschließlich der sprachlichen Ausformulierung eines bereits korrekt berechneten Ergebnisses.
- Erzwingt eine klare Trennung zwischen **Rechnen/Reasoning** und **sprachlicher Ausgabe**.

### Inputs
- Pflichtparameter:
  - `tokenizer` – Tokenizer mit Chat-Template-Unterstützung.
  - `user_question: str` – ursprüngliche Nutzerfrage.
  - `final_value: Any` – bereits bestimmtes, finales Ergebnis (nicht erneut zu interpretieren).

### Prompt-Design
- System-Nachricht definiert eine stark eingeschränkte Rolle:
  - keine erneuten Berechnungen,
  - keine Tool-Aufrufe,
  - keine strukturierte Ausgabe (z. B. JSON).
- Instruiert explizit ein festes Antwortformat, um Konsistenz und Vergleichbarkeit sicherzustellen.
- User-Nachricht liefert Frage und Endergebnis getrennt und explizit markiert.

### Outputs
- Rückgabewert:
  - `str` – vollständig gerenderter Chat-Prompt, kompatibel mit Chat-basierten LLMs.

In [25]:
def build_final_answer_prompt(
    tokenizer,
    user_question: str,
    final_value: Any,
) -> str:
    messages = [
        {
            "role": "system",
            "content": (
                        "Du bist ein sprachlicher Antwort-Renderer.\n"
                        "Das Rechenergebnis ist bereits korrekt bestimmt.\n"
                        "Du darfst NICHT neu rechnen und KEINE Tools verwenden.\n"
                        "Du darfst KEIN JSON ausgeben.\n\n"
                        "AUFGABE:\n"
                        "- Formuliere eine vollständige, natürliche Antwort auf die Nutzerfrage.\n"
                        "- Verwende exakt das gegebene Endergebnis.\n\n"
                        "FORMAT:\n"
                        "Die Antwort soll lauten:\n"
                        "\"Die Antwort auf <Frage> = <Ergebnis>\""
                    ),
        },
        {
            "role": "user",
            "content": (
                f"User-Frage:\n{user_question}\n\n"
                f"Finales Ergebnis (berechnet):\n{final_value}\n\n"
                "Gib die Antwort jetzt aus."
            ),
        },
    ]

    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

## Agentische Orchestrierung bis zur finalen Antwort

### Funktion
- Implementiert die zentrale Orchestrierungs- und Kontrollschleife eines agentischen Workflows.
- Trennt den Gesamtprozess explizit in:
  - **Planung** (LLM #1 erzeugt einen strukturierten Plan),
  - **Ausführung** (Tools werden Schritt für Schritt gemäß Plan ausgeführt),
  - **Auflösung** (Final-Spezifikation wird aus Tool-Ergebnissen abgeleitet),
  - **Finalisierung** (LLM #2 formuliert die Nutzerantwort).

### Inputs
- Pflichtparameter:
  - `user_input: str` – ursprüngliche Nutzerfrage.
  - `system_prompt: str` – Systemkontext/Regeln für den Agenten.
  - `tokenizer` – Tokenizer-Objekt zur Prompt-Konstruktion.
  - `call_planner_llm_fn: Callable[[str], str]` – LLM-Funktion für Planerstellung (liefert Text/JSON).
  - `call_final_llm_fn: Callable[[str], str]` – LLM-Funktion zur finalen Antwortformulierung.
  - `call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]]` – asynchrone Tool-Schnittstelle.
- Optionale Parameter:
  - `history_pairs` – (User, Assistant)-Paare als Gesprächskontext für Planning/RAG.
  - `tool_history` – bisherige Tool-Aufrufe und Ergebnisse (für Nachvollziehbarkeit & Replanning).

### Outputs
- Rückgabewert:
  - `str` – finaler Antworttext (typischerweise durch LLM #2 formuliert).
  - Fallback-Pfad: bei fehlender `final`-Spezifikation im Plan wird ein definierter Fehlerstring zurückgegeben.

### Interne Phasen
- **Phase 1: Einmal planen**
  - Baut einen Prompt aus Systemkontext, Nutzerfrage, Verlauf und Tool-Historie.
  - Lässt LLM #1 einen Plan erzeugen und parst ihn in eine strukturierte Repräsentation.
- **Phase 2: Plan ausführen**
  - Bestimmt iterativ den nächsten noch nicht ausgeführten Schritt.
  - Führt Tool-Schritte aus, sammelt Ergebnisse und erweitert die Tool-Historie.
- **Phase 3: Final aus Plan auflösen**
  - Löst die im Plan definierte finale Ausgabe anhand der Tool-Ergebnisse und des User-Kontexts auf.
  - Extrahiert einen skalaren Endwert, der als Grundlage für die Antwort dient.
- **Phase 4: Einmal final formulieren**
  - Baut einen finalen Prompt und lässt LLM #2 die Nutzerantwort erzeugen.
  - Schneidet Whitespace und gibt den Text zurück.

In [26]:
async def agentic_run_to_final_answer(
    *,
    user_input: str,
    system_prompt: str,
    tokenizer,
    call_planner_llm_fn: Callable[[str], str],  # LLM #1: JSON Plan
    call_final_llm_fn: Callable[[str], str],    # LLM #2: Final text
    call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]],
    history_pairs: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
) -> str:
    """
    Hauptschleife:
    - plant mit dem LLM,
    - führt Step für Step Tools aus,
    - liefert am Ende das finale Ergebnis
    """
    if tool_history is None:
        tool_history = []
    if history_pairs is None:
        history_pairs = []

    tool_results: Dict[str, Any] = {}
    executed_step_indices: Set[int] = set()
    user_ctx = {"raw": user_input}

    # -------------------------
    # Phase 1: EINMAL planen
    # -------------------------
    #<<Bitte ergänzen>>

    # -------------------------
    # Phase 2: Plan komplett ausführen
    # -------------------------
    #<<Bitte ergänzen>>

    # -------------------------
    # Phase 3: Final aus Plan auflösen
    # -------------------------
    if plan.get("final") is None:
        # Kein Replanning gewünscht -> harter Fail
        return "invalid_expression"

    final_spec = plan["final"]
    final_resolved = agentic_resolve_argument(final_spec, tool_results, user_ctx)
    final_value = _extract_scalar_result(final_resolved)

    # -------------------------
    # Phase 4: EINMAL final formulieren (LLM #2)
    # -------------------------
    final_prompt = build_final_answer_prompt(
        tokenizer=tokenizer,
        user_question=user_input,
        final_value=final_value,
    )

    final_text = call_final_llm_fn(final_prompt).strip()

    # Optional: wenn das LLM doch labert, kannst du es “hart” machen:
    # return str(final_value)
    return final_text

## MCP-Tool-Aufruf über agentische Schnittstelle

### Funktion
- Stellt eine asynchrone Brücke zwischen dem Agenten und externen MCP-Tools bereit.
- Kapselt Initialisierung, Tool-Aufruf und Ergebnis-Normalisierung in einer einheitlichen Funktion.
- Ziel ist eine robuste, agentenfreundliche Rückgabeform unabhängig vom konkreten Tool.

### Ablauf
- Gibt zu Debug-Zwecken Tool-Name und Argumente aus.
- Baut eine HTTP-basierte, streamfähige Verbindung zum MCP-Server auf.
- Initialisiert eine MCP-Session und ruft das gewünschte Tool mit Parametern auf.
- Protokolliert Typ und Repräsentation des rohen Tool-Ergebnisses zur Analyse.

### Ergebnis-Normalisierung
- Erzeugt ein standardisiertes Rückgabe-Dictionary mit:
  - `isError`: Kennzeichnung, ob der Tool-Aufruf fehlschlug.
  - `data`: normalisierte Nutzdaten.
- Bevorzugt strukturierte Inhalte (`structuredContent`), falls vorhanden.
- Fällt andernfalls auf Textinhalte zurück und aggregiert diese zu einem String.
- Stellt sicher, dass der Agent immer mit einer konsistenten Datenstruktur weiterarbeiten kann.

### Outputs
- Rückgabewert:
  - `dict` mit den Schlüsseln:
    - `isError`: bool
    - `data`: strukturierte Daten oder zusammengeführter Textinhalt

In [27]:
from mcp.types import TextContent  # ggf. anpassen

async def agentic_call_mcp_tool(tool_name: str, args: dict) -> dict:
    print(f"[DEBUG] agentic_call_mcp_tool: {tool_name}({args})")
    async with streamable_http_client(MCP_URL) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            raw_result = await session.call_tool(tool_name, args)
            print(f"[DEBUG] Raw tool result type: {type(raw_result)}")
            print(f"[DEBUG] Raw tool result repr: {repr(raw_result)[:400]}")

            # Normalisieren
            result_dict: dict = {
                "isError": getattr(raw_result, "isError", False),
            }

            # strukturierte Inhalte bevorzugen
            if getattr(raw_result, "structuredContent", None) is not None:
                result_dict["data"] = raw_result.structuredContent
            else:
                # Fallback: Text aus content zusammenbauen
                texts = []
                for c in getattr(raw_result, "content", []) or []:
                    if isinstance(c, TextContent):
                        texts.append(c.text)
                result_dict["data"] = "\n".join(texts)

            return result_dict

## Agentischer Lauf bis zur finalen Antwort

### Kontext
- Demonstriert einen vollständigen agentischen Durchlauf von einer Nutzerfrage bis zur finalen, ausformulierten Antwort.
- Der Ablauf kombiniert Planungs-LLM, Ausführungs-LLM und Tool-Aufrufe in einer kontrollierten Pipeline.

### Ablauf
- Definiert eine einfache Nutzerfrage als String (`user_input`).
- Initialisiert einen leeren Verlauf für agentisches Reasoning.
- Ruft eine zentrale Orchestrierungsfunktion auf, die:
  - einen **Planner-LLM** zur Strukturierung des Vorgehens nutzt,
  - einen **Final-LLM** zur deterministischen Generierung der Endantwort verwendet,
  - optional externe Tools über eine einheitliche Schnittstelle aufruft,
  - und den gesamten Dialog- und Tool-Verlauf verwaltet.

### Eingesetzte Komponenten
- `call_planner_llm_fn`: Wrapper für ein LLM mit größerem Token-Budget zur Planung.
- `call_final_llm_fn`: Wrapper für ein LLM mit kleinerem Token-Budget und deterministischen Einstellungen zur Antwortformulierung.
- `call_tool_fn`: Abstraktion für Tool-Aufrufe (z. B. Rechnen, Retrieval, externe Services).
- `history_pairs` / `tool_history`: explizite Übergabe von Kontext, um den Lauf reproduzierbar und kontrollierbar zu halten.

### Output
- `final_answer` enthält ausschließlich die finale, für den Nutzer bestimmte Antwort.
- Die Ausgabe trennt klar zwischen interner Agentenlogik und externer Nutzerkommunikation.

### Typische Verwendung
- Evaluierung agentischer Architekturen.
- Debugging von Planner-/Executor-Aufteilungen.
- Reproduzierbare Experimente mit Tool-augmented LLMs in Jupyter-Notebooks.

In [28]:
user_input = f"Was ist 6.0 + 4.0 * 8.0?"
history_agentic = []

final_answer = await agentic_run_to_final_answer(
        user_input=user_input,
        system_prompt=agent_system_prompt,
        tokenizer=tokenizer,
        call_planner_llm_fn=lambda p: llama_chat(p, max_new_tokens=512),
        call_final_llm_fn=lambda p: llama_chat(p, max_new_tokens=64, temperature=0.0),
        call_tool_fn=agentic_call_mcp_tool,
        history_pairs=[],
        tool_history=None,
    )

print("Finale Antwort an den User:")
print(final_answer)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
[01/31/26 10:42:23] INFO     Created new          ]8;id=224399;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=800100;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             3c2c495253474e2f910a                               
                             4f907c34ba32                                       
                    INFO     Processing request of type            ]8;id=23499;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=626004;file:///home/simon/.virtualenvs

[PHASE 1] planner output:
{
  "steps": [
    {
      "description": "Multiplikation von 4.0 und 8.0",
      "tools": [
        {
          "id": "mul_1",
          "name": "multiplikation",
          "arguments": {
            "a": 4.0,
            "b": 8.0
          },
          "replace_target": "4.0 * 8.0"
        }
      ]
    },
    {
      "description": "Addition von 6.0 und 32.0",
      "tools": [
        {
          "id": "add_1",
          "name": "addition",
          "arguments": {
            "a": 6.0,
            "b": {"$ref": "mul_1.result"}
          },
          "replace_target": "6.0 + 4.0 * 8.0"
        }
      ]
    }
  ],
  "final": {"$ref": "add_1.result"}
}

[DEBUG] trimmed JSON candidate: {
  "steps": [
    {
      "description": "Multiplikation von 4.0 und 8.0",
      "tools": [
        {
          "id": "mul_1",
          "name": "multiplikation",
          "arguments": {
            "a": 4.0,
            "b": 8.0
          },
          "replace_target": "4.0 